# Задача оттока клиентов банка

Цель:
    Необходимо решить задачу предсказания оттока клиентов банка. А именно, используя данные транзакций за 6 месяцев, требуется построить алгоритм, предсказывающий вероятность оттока клиента в следующие 6 месяцев. Особенностью задачи является то, что в рамках тренировочных данных для обучения, участникам передается не только метка, соответствующая тому, что клиент “уйдёт в отток”, но и время до его последней транзакции. 

Метрика:
    Метрика соревнования — CI, также известная как Concordance Index или (Harrel's) C-index. Для тех, кто впервые сталкивается с этой метрикой, на нее стоит смотреть как на обобщение ROC-AUC для Time-to-Event задач.

Глоссарий

clients.csv – базовая информация про клиентов:

    user_id – ID клиента банка
    report — номер одного из 12 отчетов, в который попал этот клиент
    employee_count_nm – информация о работодателе клиента: число сотрудников в компании (осторожно, есть пропуски)
    bankemplstatus – информация о том, является ли клиент сотрудников банка
    customer_age – обфусцированный возраст клиента (до одной из 4 групп 0..3)

train.csv – тренировочные данные с целевой переменной про клиентов:

    user_id – ID клиента банка
    target – метка оттока клиента (целевой класс события оттока это 1)
    time – через сколько дней случится последняя транзакция клиента (можно использовать как survival time в Time-to-Event анализе)

report_dates.csv – информация о датах отчетов:

    report – порядковый номер отчета
    report_dt — дата отчета 

 transactions.csv.zip– архив с транзакциями клиентов банка:

    user_id – ID клиента банка
    mcc_code — mcc код транзакции
    currency_rk – валюта транзакции
    transaction_dttm – дата и время совершения операции
    transaction_amt – сумма в валюте транзакции

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/vtb-competishions/train.csv')
train.head()

In [ ]:
clients = pd.read_csv('../input/vtb-competishions/clients.csv')
clients

In [ ]:
report_dates = pd.read_csv('../input/vtb-competishions/report_dates.csv', parse_dates=['report_dt'])
report_dates

In [ ]:
transactions = pd.read_csv('../input/vtb-competishions/transactions.csv',parse_dates=['transaction_dttm'])
transactions

In [ ]:
client_report = pd.merge(clients,
               report_dates,
               on = 'report',
               how = 'left')
client_report

In [ ]:
client_report_transaction = pd.merge(client_report,
               transactions,
               on = 'user_id',
               how = 'left')
client_report_transaction

In [ ]:
train['target'].value_counts()
sns.set_palette('Blues',3)
plt.pie(train['target'].value_counts(), labels = ['Not Exited','Exited'], autopct='%.2f%%')
plt.show()

In [ ]:
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1


In [ ]:
client_report_transaction.columns
client_report_transaction.columns = client_report_transaction.columns.astype(str)

In [ ]:
count_trans = client_report_transaction['user_id'].value_counts().to_frame().reset_index()
count_trans

In [ ]:
#сумма транзакций
sum_transaction = client_report_transaction.groupby(['user_id'])[['transaction_amt']].sum().reset_index()
sum_transaction = sum_transaction.rename(columns={"transaction_amt":"sum_trans"}).sort_values(["user_id"]).reset_index(drop=True)
sum_transaction

In [ ]:
#сумма положительных транзакций
sum_transaction_p = client_report_transaction[client_report_transaction['transaction_amt'] > 0 ].groupby('user_id')['transaction_amt'].sum().to_frame().reset_index()
sum_transaction_p = sum_transaction_p.rename(columns={"transaction_amt":"sum_trans_p"}).sort_values(["user_id"]).reset_index(drop=True)

sum_transaction_p

In [ ]:
# сумма отрицательных транзакций
sum_transaction_n = client_report_transaction[client_report_transaction['transaction_amt'] < 0 ].groupby('user_id')['transaction_amt'].sum().to_frame().reset_index().rename(columns={"transaction_amt":"sum_trans_n"}).sort_values(["user_id"]).reset_index(drop=True)

sum_transaction_n

In [ ]:
client_report_transaction['diff_days'] = (client_report_transaction['report_dt']-client_report_transaction['transaction_dttm']).dt.days

In [ ]:
# Максимальная дата до отчета, по клиентам
max_day_trans = client_report_transaction.groupby(['user_id'])[['diff_days']].max().reset_index().rename(columns={"diff_days":"max_day_trans"}).sort_values(["user_id"]).reset_index(drop=True)
max_day_trans

In [ ]:
min_day_trans = client_report_transaction.groupby(['user_id'])[['diff_days']].min().reset_index().rename(columns={"diff_days":"min_day_trans"}).sort_values(["user_id"]).reset_index(drop=True)
min_day_trans

In [ ]:
clients=merge_by_concat(clients, count_trans, ['user_id'])

clients=merge_by_concat(clients, sum_transaction, ['user_id'])

clients=merge_by_concat(clients, sum_transaction_n, ['user_id'])

clients=merge_by_concat(clients, sum_transaction_p, ['user_id'])

clients=merge_by_concat(clients, max_day_trans, ['user_id'])

clients=merge_by_concat(clients, min_day_trans, ['user_id'])
clients

In [ ]:
clients['employee_count_nm'].unique()

In [ ]:
employee_count_nm_min_max = clients['employee_count_nm'].str.split(" ", n=3, expand=True)
conditions = [
    employee_count_nm_min_max[0] == 'ОТ',
    employee_count_nm_min_max[0] == 'БОЛЕЕ',
    employee_count_nm_min_max[0] == 'ДО',
    employee_count_nm_min_max[0] == 'None',
    ]
choices_for_min = [employee_count_nm_min_max[1], employee_count_nm_min_max[1], 0,0]
choices_for_max = [employee_count_nm_min_max[3], 0, employee_count_nm_min_max[1],0]
employee_count_nm_min_max['min_employee_count_nm'] = np.select(conditions, choices_for_min)
employee_count_nm_min_max['max_employee_count_nm'] = np.select(conditions, choices_for_max)
employee_count_nm_min_max
employee_count_nm_min_max.drop([0,1,2,3],axis =1, inplace = True)
#clients.drop(columns=["employee_count_nm"], inplace=True)
#clients
employee_count_nm_min_max


In [ ]:
clients = pd.concat([clients, employee_count_nm_min_max], axis=1)
clients.drop(columns=["employee_count_nm"], inplace=True)
clients

Моделирование

In [ ]:
clients.info()

In [ ]:
clients['min_employee_count_nm'] = clients['min_employee_count_nm'].astype('int64')
clients['max_employee_count_nm'] = clients['max_employee_count_nm'].astype('int64')
clients.columns

In [ ]:
categorical_col = ['report','bankemplstatus','customer_age']
numeric_col = ['count','sum_trans', 'sum_trans_n', 'sum_trans_p', 'max_day_trans','min_day_trans', 'min_employee_count_nm', 'max_employee_count_nm']

In [ ]:
train_full = clients.merge(train, on='user_id', how='outer')
train_full

In [ ]:
train_full.isna().sum()

In [ ]:
train_full['sum_trans_n'] = train_full['sum_trans_n'].fillna(0)
train_full['sum_trans_p'] = train_full['sum_trans_p'].fillna(0)
train_full.isna().sum()

In [ ]:
features_targets = ['user_id', 'report', 'bankemplstatus', 'customer_age', 'count',
       'sum_trans', 'sum_trans_n', 'sum_trans_p', 'max_day_trans',
       'min_day_trans', 'min_employee_count_nm', 'max_employee_count_nm']
train_full

In [ ]:
train_full['time'] = train_full['time'].fillna(-1)
train_full['time'] = train_full['time'].astype(np.int32)

In [ ]:
train_full['target'] = train_full['target'].fillna(-1)
train_full['target'] = train_full['target'].astype(np.int8)

In [ ]:
X = train_full[train_full['time']!=-1][features_targets].copy()
y = train_full[train_full['time']!=-1][['target', 'time']].copy()

In [ ]:
y['target'] = y['target'].astype(bool)

In [ ]:
aux = [(e1,e2) for e1,e2 in  np.array(y)]

In [ ]:
from sklearn.model_selection import train_test_split
y = np.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

y_train


In [ ]:
!pip install lifelines

In [ ]:
from lifelines import CoxPHFitter


cph = CoxPHFitter()
cph.fit(train_full, duration_col='time', event_col='target')

cph.print_summary() 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc 
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay




pipe = Pipeline([('scaler', StandardScaler()),
                ('model_lr', LogisticRegression())])

pipe.fit(X_train, y_train)


# grid_params = {'model_lr__penalty': ['l1', 'l2'],
#               'model_lr__C': [1e-5, 0.0001, 0.001, 0.01]}

# search = GridSearchCV(pipe, param_grid=grid_params, cv = 2, scoring='roc_auc')

# search.fit(X,y)

# best_model = search.best_estimator_
# best_model.fit(X_train,y_train)

# RocCurveDisplay.from_estimator(best_model, X_test,y_test)
fpr, tpr, thresholds = roc_curve(y_test, best_model.predict_proba(X_test)[:, 1])
round(auc(fpr, tpr),3)

In [ ]:
X_pred = train_full[train_full['time']==-1][features_targets].copy()
X_pred